# 收集数据部分

## collect_data.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/collect_data.py)

### 1 get_output()

原本的收集数据的策略是采取独热编码的，现在变成一个前半部分由独热编码（攻防垫跳和其他）组成和后半部分由道具使用信息（伤药葫芦、BUFF）和位移信息（前后左右）组成的列表  
目的是期望这些新增的信息在将来能够缓解数据不平衡，不过当前参加训练的依然是前面独热编码部分（通过列表切片的方式取出独热编码部分），新增的部分暂时没有用到

~~~python
def get_output(keys):

    output = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # 对参与训练的按键信息进行独热编码
    # 攻击、防御、垫步、跳跃和使用道具不能同时进行（指0.1秒内），但是可以和移动同时进行
    if   'J' in keys:
        output[0] = 1    # 等同于[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    elif 'K' in keys:
        output[1] = 1    # 等同于[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    elif 'LSHIFT' in keys:
        output[2] = 1    # 等同于[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    elif 'SPACE' in keys:
        output[3] = 1    # 等同于[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    elif 'R' in keys:
        output[5] = 1    # 等同于[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]    不参与训练
        output[4] = 1    # 等同于[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
    else:
        output[4] = 1    # 等同于[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

    # 不能同时前后移动
    if   'W' in keys:
        output[6] = 1    # 等同于[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]    不参与训练
    elif 'S' in keys:
        output[7] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]    不参与训练

    # 不能同时左右移动
    if   'A' in keys:
        output[8] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]    不参与训练
    elif 'D' in keys:
        output[9] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]    不参与训练

    return output
~~~

### 2 class Data_collection

~~~python
def __init__(self, target):
    self.target = target    # 目标
    self.dataset = list()    # 保存数据的容器
    self.save_path = os.path.join('The_battle_memory', self.target)    # 保存的位置
    if not os.path.exists(self.save_path):    # 确保保存的位置存在
        os.mkdir(self.save_path)

    self.step = 0    # 计步器
~~~

#### 1.2.1 保存数据方法

保存数据，统一名称为 '**training_data-** 序号 **.npy**'，例如 training_data-123.npy  
从序号为1开始检测文件名是否存在于保存路径，直到检测到不存在就保存，主要是为了防止文件名重复不小心把原本的文件覆盖了

~~~python
def save_data(self):
    n = 1
    while True:    # 直到找到保存位置并保存就 break
        filename = f'training_data-{n}.npy'
        save_path = os.path.join(self.save_path, filename)
        if not os.path.exists(save_path):    # 没有重复的文件名就执行保存并退出
            print(save_path)
            np.save(save_path, self.dataset)
            break
        n += 1
~~~

#### 1.2.2 收集数据方法

这部分代码的作用是：记录当前这个循环开始的时间，不断检测按键信息（核心部分也要用到这个按键信息），按 'T' 取消暂停状态开始收集数据，最后按 'P' 退出循环，保存数据

~~~python
def collect_data(self):

    paused = True
    while True:
        last_time = time.time()
        keys = key_check()
        if paused:
            if 'T' in keys:
                paused = False
        else:

            self.step += 1

            """
            核心部分，看下面
            """

            if 'P' in keys:    # 结束
                self.save_data()    # 保存数据
                break
~~~

**核心部分**  

~~~python
screen = get_screen()    # 获取屏幕图像
if not (np.sum(screen == 0) > 97200):    # 270 * 480 * 3 / 4 = 97200 ，当图像有1/4变成黑色（像素值为0）的时候停止暂停收集数据
    action_list = get_output(keys)    # 获取按键输出列表
    self.dataset.append([screen, action_list])    # 图像和输出打包在一起，保证一一对应

# 降低数据采集的频率，周期为0.1秒
T = 0.1
t = T-(time.time()-last_time)
if t > 0:
    time.sleep(t)
~~~

- 记录数据的条件是非黑屏  
- 数据采集降频的目的主要是提高 action 与 reward 的对应程度，其次是缓解数据不平衡和
    - reward 是执行当前动作后观测状态变化所得到的反馈，如果采集频率过高，会导致状态还没变化就计算了 reward ，这样就无法得到正确的 reward 。action 与 reward 不对应会导致训练出现严重的后果甚至训练结果是无效的。
        - 注：当前设置的采集频率 f=10 只是当前的标准，如果未来发现更合适的采集频率，可能会更改此参数
    - 数据采集降频理论上不能改变各类动作的数据占比，但能够缩小了数据量的差距
        - 按键检测原理详见[了解项目基础部分是如何工作的](https://github.com/ricagj/pysekiro_with_RL/blob/main/How_it_works.ipynb)

### 3 数据预览

可以先看这个演示 https://github.com/ricagj/pysekiro/blob/main/imgs/data_preview.gif

**注：以下展示的是我本地的数据集，我没有上传，所以要运行以下代码，请先自己收集至少一个数据集**

- Data_quality (存放数据集对应的reward曲线)
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.png （第一个战斗数据的reward曲线）
-
- The_battle_memory
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.npy （第一个战斗数据）

In [ ]:
import os

import cv2
import numpy as np
import pandas as pd

target = 'Genichiro_Ashina' # 苇名弦一郎
# target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心

path = os.path.join('The_battle_memory', target, f'training_data-{1}.npy')
if os.path.exists(path):
    data = np.load(path, allow_pickle=True)
    print(f'当前数据集所包含数据量：{data.shape[0]}, 每个数据都有 {data.shape[1]} 个数据，分别是图像数据和按键数据')
else:
    print('数据集不存在，以下代码无法运行')

In [ ]:
screen = data[0][0]    # 选取了第一个数据的图像部分
# print('图像', screen)
print('形状', screen.shape)

In [ ]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)
print('目前的训练只使用前五个，即独热编码部分  ', action_value[:5])

In [ ]:
# 统计动作分布
def motionCounts(Y):

    columns=['攻击', '弹反', '垫步', '跳跃', '无键', '道具', '向前', '向后', '向左', '向右']
    df = pd.DataFrame(Y, columns=columns, dtype=np.uint8)
    
    total = len(df)

    for motion in columns:
        motion_count = len(df)-df[motion].value_counts()[0]
        print(f'motion: {motion:>3}, 记录到的次数: {motion_count:>5}')

Y = np.array([i[1] for i in data])
motionCounts(Y)

In [ ]:
from pysekiro.get_vertices import roi

print()
print('以视频的形式展示数据。', '要你自己运行才看得见视频')
Remaining = len(data)
for screen, action_value in data:
    action = np.argmax(action_value[:5])

    if   action == 0:
        a = '攻击'
    elif action == 1:
        a = '弹反'
    elif action == 2:
        a = '垫步'
    elif action == 3:
        a = '跳跃'
    elif action == 4:
        a = '其他'

    cv2.imshow('screen', screen)
    cv2.imshow('roi', roi(screen, x=140, x_w=340, y=30, y_h=230))
    cv2.waitKey(10)

    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{a} {action_value}', end='') # end='\n'
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

In [ ]:
# 获取数据集对应的reward曲线
from pysekiro.Agent import get_data_quality

get_data_quality()

# 模型训练部分

## learn_offline.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_offline.py)

~~~python
# 依次读取训练集进行离线学习
for i in range(start, end+1):

    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)

    if os.path.exists(data_path):    # 确保数据集存在

        # 加载数据集
        data = np.load(data_path, allow_pickle=True)

        """
        学习部分，看下面
        """
        
        sekiro_agent.save_evaluate_network()    # 这个数据学习完毕，保存网络权重
        sekiro_agent.reward_system.save_reward_curve(save_path=reward_curve_save_path)    # 绘制 reward 曲线并保存

    else:
        print(f'{filename} does not exist ')
~~~

**学习部分**  
主要思路：获取 (状态S、动作A、奖励R 和 新状态S')，存储以供经验回放，然后目标网络和评估网络根据各自设置的更新频率更新自身网络参数  

~~~python
sekiro_agent.reward_system.cur_status = get_status(dataset[0][0])    # 设置初始状态
for step in range(len(data)-1):

    # ---------- (S, A, R, S') ----------

    screen = data[step][0]               # 状态S
    action = np.argmax(data[step][1][:5])    # 动作A
    next_screen = data[step+1][0]        # 新状态S'
    reward = sekiro_agent.reward_system.get_reward(get_status(next_screen))    # 奖励R

    # ---------- store ----------

    # 集齐 (S, A, R, S')，开始存储
    sekiro_agent.replayer.store(
        cv2.resize(roi(screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)),    # 截取感兴趣区域并图像缩放
        action,
        reward,
        cv2.resize(roi(next_screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT))    # 截取感兴趣区域并图像缩放
    )    # 存储经验

    # ---------- learn ----------

    sekiro_agent.step = step + 1
    sekiro_agent.learn()
~~~

## learn_online.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_online.py)

由 collect_data.py 中 Data_collection.collect_data() 变化而来

按 'T' 开始，然后等待按 'P' 结束
~~~python
if save_path:    # 判断是训练模式还是测试模式
    train = True
else:
    train = False

paused = True
print("Ready!")

step = 0    # 计步器

while True:

    last_time = time.time()
    keys = key_check()
    if paused:
        screen = get_screen()    # 首个 状态S，但是在按 'T' 之前，它会不断更新
        if 'T' in keys:
            paused = False
            print('\nStarting!')
    else:

        step += 1
        
        """
        核心部分，看下面
        """

        # 降低数据采集的频率，两次采集的时间间隔为0.1秒
        t = 0.1-(time.time()-last_time)
        if t > 0:
            time.sleep(t)

        if 'P' in keys:
            if train:
                sekiro_agent.save_evaluate_network()    # 学习完毕，保存网络权重
            sekiro_agent.reward_system.save_reward_curve(save_path=reward_curve_save_path)    # 绘制 reward 曲线并保存在当前目录
            break
~~~

**核心部分**  
- 按 'T' 之后，获得首个状态S，然后依次获得 动作A、奖励R 和 新状态S'，再进入第二个轮回  
- 第二个轮回开始
    1. 原本的 **新状态S'** 变成 **当前状态S**
    2. 由 **当前状态S** 选取 **动作A**
    3. 由 **当前状态S** 获取 **奖励R**
    4. 观测 **新状态S'**
    - 进入下一个轮回
- 这样做的目的是
    1. 集齐 (S, A, R, S')
    2. 保证 状态S 和 新状态S' 连续

~~~python
# ---------- (S, A, R, S') ----------

action = sekiro_agent.choose_action(screen, train)    # 动作A
next_screen = get_screen()    # 新状态S'
reward = sekiro_agent.reward_system.get_reward(get_status(next_screen))    # 奖励R

# ---------- 下一个轮回 ----------

screen = next_screen    # 状态S

"""
学习部分，看下面
"""
~~~

**学习部分**  
~~~python
if train:
    if not (np.sum(screen == 0) > 97200):    # 270 * 480 * 3 / 4 = 97200 ，当图像有1/4变成黑色（像素值为0）的时候停止暂停存储数据

        # ---------- store ----------

        # 集齐 (S, A, R, S')，开始存储
        sekiro_agent.replayer.store(
            cv2.resize(roi(screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)),    # 截取感兴趣区域并图像缩放
            action,
            reward,
            cv2.resize(roi(next_screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT))    # 截取感兴趣区域并图像缩放
        )    # 存储经验

        # ---------- learn ----------

        sekiro_agent.step += 1
        sekiro_agent.learn()
~~~

# DQN部分

# Agent.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/Agent.py)

[参考来源1](https://github.com/ZhiqingXiao/rl-book/blob/master/chapter10_atari/BreakoutDeterministic-v4_tf.ipynb)  
[参考来源2](https://mofanpy.com/tutorials/machine-learning/reinforcement-learning/DQN3/)  

## 1 class RewardSystem

计算 reward 的原理
~~~python
# 约束状态值的上下限，防止异常值和特殊值的影响。
def limit(value, lm1, lm2):

    if value < lm1:
        return lm1
    elif value > lm2:
        return lm2
    else:
        return value

# 自身状态的计算方法：(下一个的状态 - 当前的状态) * 正负强化权重
s1 = (self.next_status[0] - self.cur_status[0]) *  1    # 自身生命
s2 = (self.next_status[1] - self.cur_status[1]) * -1    # 自身架势

# 目标状态的计算方法：约束上下限(下一个的状态 - 当前的状态) * 正负强化权重
t1 = limit((self.next_status[2] - self.cur_status[2]), -100,   0) * -1    # 目标生命
t2 = limit((self.next_status[3] - self.cur_status[3]),  -20, +20) *  1    # 目标架势

# 分开生命值和架势并赋予这样的权重是为了降低生命值状态变化的影响
reward = 0.1 * (s1 + t1) + 0.9 * (s2 + t2)
~~~

## 2 class DQNReplayer

这个不是我写的的  
[来源](https://github.com/ZhiqingXiao/rl-book/blob/master/chapter10_atari/BreakoutDeterministic-v4_tf.ipynb)

## 3 class Sekiro_Agent

~~~python
def __init__(
    self,
    n_action = n_action, 
    gamma = 0.99,
    batch_size = 128,
    replay_memory_size = 20000,
    epsilon = 1.0,
    epsilon_decrease_rate = 0.999,
    update_freq = 100,
    target_network_update_freq = 300,
    model_weights = None,
    save_path = None
):
    self.n_action = n_action    # 动作数量

    self.gamma = gamma    # 奖励衰减

    self.batch_size = batch_size                    # 样本抽取数量
    self.replay_memory_size = replay_memory_size    # 记忆容量

    self.epsilon = epsilon                                # 探索参数
    self.epsilon_decrease_rate = epsilon_decrease_rate    # 探索衰减率

    self.update_freq = update_freq    # 训练评估网络的频率
    self.target_network_update_freq = target_network_update_freq    # 更新目标网络的频率

    self.model_weights = model_weights    # 指定读取的模型参数的路径
    self.save_path = save_path            # 指定模型权重保存的路径
    if not self.save_path:
        self.save_path = 'tmp_weights.h5'

    self.evaluate_net = self.build_network()    # 评估网络
    self.target_net = self.build_network()      # 目标网络
    self.reward_system = RewardSystem()                     # 奖惩系统
    self.replayer = DQNReplayer(self.replay_memory_size)    # 经验回放

    self.step = 0    # 计步
~~~

### 3.1 评估网络和目标网络的构建方法
~~~python
def build_network(self):
    model = MODEL(ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT,
        outputs = self.n_action,
        model_weights = self.model_weights
    )
    return model
~~~

### 3.2 行为选择与执行方法
~~~python
def choose_action(self, screen, train):
    if train:
        r = np.random.rand()
    else:
        r = 1.01    # 永远大于 self.epsilon

    # train = True 开启探索模式
    if r < self.epsilon:
        self.epsilon *= self.epsilon_decrease_rate    # 逐渐减小探索参数, 降低行为的随机性
        action = np.random.randint(self.n_action)

    # train = False 直接进入这里
    else:
        screen = cv2.resize(roi(screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)).reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
        q_values = self.evaluate_net.predict(screen)[0]
        action = np.argmax(q_values)

    # 执行动作
    act(action)

    return action
~~~

### 3.3 学习方法
~~~python
def learn(self):

    if self.step >= self.batch_size and self.step % self.update_freq == 0:    # 更新评估网络

        if self.step % self.target_network_update_freq == 0:    # 更新目标网络
            print(f'step:{self.step:>4}, current_cumulative_reward:{self.reward_system.current_cumulative_reward:>5.3f}, memory:{self.replayer.count:7>}')
            self.update_target_network() 

        # 经验回放
        screens, actions, rewards, next_screens = self.replayer.sample(self.batch_size)

        screens = screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
        next_screens = next_screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)

        # 计算回报的估计值
        q_next = self.target_net.predict(next_screens)
        q_target = self.evaluate_net.predict(screens)
        q_target[range(self.batch_size), actions] = rewards + self.gamma * q_next.max(axis=-1)

        self.evaluate_net.fit(screens, q_target, verbose=0)
~~~

### 3.4 更新目标网络权重方法
~~~python
def update_target_network(self):
    self.target_net.set_weights(self.evaluate_net.get_weights())
~~~

### 3.5 保存评估网络权重方法
~~~python
def save_evaluate_network(self):
    self.evaluate_net.save_weights(self.save_path)
~~~